# Personalising Spotify through a recommendation engine

## Imports

In [612]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

pd.options.mode.chained_assignment = None
#gets rid of annoying false-positive warning


In [613]:
%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [614]:
# import sqlite3
# try:
#     conn = sqlite3.connect("million_songs.db")
# except Exception as e:
#     print(e)

# # Now in order to read in pandas dataframe we need to know table name
# cursor = conn.cursor()
# songs_df = pd.read_sql_query('SELECT * FROM FEATURES_2', conn)
# many_genres_df = pd.read_sql_query('SELECT * FROM TRAINING_DB_2', conn)
# conn.close()

# all_songs_df = pd.read_csv('data/songs.csv').head(100) #making it a small number so i can manage lol

In [615]:
def generate_dataframes(library_songs, wasabia_songs):

    library_songs_df = pd.DataFrame(library_songs)
    wasabia_songs_df = pd.DataFrame(wasabia_songs)

    library_songs_df = library_songs_df.drop_duplicates(subset=['id'])
    wasabia_songs_df = wasabia_songs_df.drop_duplicates(subset=['id'])

    library_songs_length = len(library_songs_df.index)
    wasabia_songs_length = len(wasabia_songs_df.index)

    return library_songs_df, wasabia_songs_df, library_songs_length, wasabia_songs_length

In [616]:

with open('personalise_test.json') as json_file:
	test_data_json = json.load(json_file)

library_songs_df, wasabia_songs_df, library_songs_length, wasabia_songs_length = generate_dataframes(test_data_json['user_playlist'], test_data_json['wasabia'])

display(library_songs_df.head())
display(wasabia_songs_df.head())

,added_at,is_local,track,genres,release_date,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,2020-07-28T11:36:31Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[aesthetic rap, phonk]",2018-01-08,Evil Fantasy,0.855,0.796,5,-5.976,1,0.3160,0.220,0.000321,0.107,0.655,80.028,3A4TIpLeMng0Nn0N7yre4g,114129,4
1,2020-08-26T12:06:40Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[corrido, corridos tumbados, nueva musica mexi...",2020-07-17,Romance,0.612,0.484,10,-8.161,1,0.0302,0.637,0.000000,0.129,0.524,97.417,6obSwTmKqCqMIFMeqUo6Yv,274215,3
2,2020-07-28T11:36:14Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[lo-fi jazzhop, christian lo-fi]",2019-02-26,no stress,0.615,0.725,0,-6.574,1,0.0362,0.117,0.126000,0.458,0.741,90.001,7ouNolPevRZcmf7wnp0Ypu,229333,4
3,2020-07-28T11:36:59Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[indie pop, vancouver indie]",2007-05-08,Ball Cap,0.470,0.200,7,-10.854,1,0.0362,0.862,0.000015,0.151,0.220,126.927,5tLCG0E1wv5GG3eWSSVU3t,200253,4
4,2020-07-28T11:35:39Z,False,"{'album': {'images': [{'height': 640, 'url': '...",[future funk],2019-02-15,The Day of Night,0.511,0.935,7,-5.417,0,0.5770,0.150,0.730000,0.466,0.438,89.852,1lWoIYZDRuyPEHP3cdo063,74710,4


,added_at,is_local,track,genres,release_date,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[instrumental math rock, japanese jazz fusion,...",2009-09-02,Waltz For Lilly,0.269,0.662,9,-8.142,1,0.0410,0.000257,0.893,0.0799,0.557,173.257,4rwQ3Rt2Cxm9dlibRhoaTV,265320,3
1,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[british post-rock, instrumental post-rock, in...",2009-01-06,Set Guitars to Kill,0.160,0.900,0,-5.343,0,0.0608,0.000052,0.844,0.3390,0.476,97.092,7vFft9G8FTZwsBJ98a7Bft,329587,4
2,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[emo, indie rock, screamo]",2017-08-18,451,0.422,0.802,2,-5.129,0,0.0496,0.000335,0.454,0.1130,0.345,131.900,7tEGdIdX1ocggOYeOYOnIU,292880,4
3,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[instrumental math rock, instrumental rock, ma...",2007-12-08,"When the Catholic Girls Go Camping, the Nicoti...",0.319,0.750,4,-4.860,0,0.0374,0.000018,0.884,0.0863,0.380,138.112,6svSukdIhpKDFaFJNrblJ4,232987,3
4,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...",[instrumental math rock],2021-01-29,The Great Calamity,0.440,0.897,6,-5.322,0,0.0539,0.000121,0.772,0.0608,0.387,137.601,7EPCNK1WiKFbUIhoHuEqii,329000,4


In [617]:
def generate_all_dataframe(library_songs_df, wasabia_songs_df):
	all_songs_df = pd.concat([library_songs_df, wasabia_songs_df]).reset_index(drop=True)
	all_songs_df['year'] = all_songs_df['release_date'].apply(lambda x: x.split('-')[0])
	return all_songs_df.drop(['release_date', 'is_local', 'track'], axis=1)

In [618]:
all_songs_df = generate_all_dataframe(library_songs_df, wasabia_songs_df)
all_songs_df.tail()

,added_at,genres,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,year
2668,2022-04-29T15:40:01Z,"[chill guitar, fingerstyle]",Movement I - Awakening,0.213,0.436,1,-11.327,1,0.0413,0.856000,0.882000,0.1240,0.400,93.761,63y9PsKjVQWxvx4yZ4QMdL,235653,3,2012
2669,2022-04-29T15:40:01Z,"[anime rap, antiviral pop, comic]",My First Rime,0.402,0.848,4,-5.984,0,0.2280,0.074300,0.000000,0.2690,0.250,96.822,5JEXleNNHnidROnQXWsJEs,157273,4,2017
2670,2022-04-29T15:40:01Z,[surf music],The Trouble With Those Mothra Girls,0.388,0.914,1,-4.635,0,0.0371,0.000256,0.725000,0.1040,0.717,121.333,5DS76z4SSW8MWAyl9NVqMg,209747,4,2005
2671,2022-04-29T15:40:01Z,[brooklyn indie],Full Metal Black,0.399,0.989,2,-5.876,1,0.0703,0.001580,0.013000,0.0522,0.174,170.528,2MQsDtg1jb4qI5OcurHMI0,183995,4,2016
2672,2022-04-29T15:40:01Z,[symphonic power metal],Udoroth,0.561,0.950,4,-3.491,0,0.0440,0.000046,0.000002,0.1540,0.573,102.471,4hSq2xpzXIt76mqkeZyJ51,271427,4,2014


## Create weight document for genres (one-time run)

In [619]:

def path_to_genre_weights(filepath, min_size):
	with open(filepath) as fp:
		all_genres = fp.read().split('\n')

	genre_range = len(all_genres)
	genre_step = (1-min_size)/genre_range
	return np.arange(min_size, 1, genre_step), all_genres

def only_run_once(genre_scale_min):
	filepath = 'data/genres.txt'
	genre_weights, genre_names = path_to_genre_weights(filepath, min_size=genre_scale_min)
	genre_weights_df = pd.DataFrame(genre_weights, columns=['weight'], index=genre_names)
	genre_weights_df.to_csv('data/genre_weights.csv')

# only_run_once(genre_scale_min=0.5)

In [620]:
def collate_genre_stats():
    genre_weights_df = pd.read_csv('data/genre_weights.csv', index_col=0)
    #grabbin it again bc we need to use it. this code can be re-used lol
    float_cols = all_songs_df.dtypes[all_songs_df.dtypes == 'float64'].index.values
    int_cols = all_songs_df.dtypes[all_songs_df.dtypes == 'int64'].index.values
    return np.concatenate((float_cols, int_cols), axis=None), genre_weights_df

In [621]:
stat_cols, genre_weights_df = collate_genre_stats()

In [622]:
def ohe_prep(df, column): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [623]:
def create_feature_set(df, stat_cols, genre_weights_df):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        stat_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    song_genre_weights_df = (
        df['genres'].explode()
        .str.get_dummies().sum(level=0)
    )

    for genre in song_genre_weights_df.columns.values:
        song_genre_weights_df[genre] *= genre_weights_df.loc[genre].values[0]

    year_ohe = ohe_prep(df, 'year') * 0.5

    #scale float columns
    floats = df[stat_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([song_genre_weights_df, floats_scaled, year_ohe, df['added_at']], axis = 1).sort_values('added_at',ascending = False)
    # final = pd.concat([song_genre_weights_df, floats_scaled, df['added_at']], axis = 1).sort_values('added_at',ascending = False)
    
    final['id']=df['id'].values
    
    return final.reset_index(drop=True)

In [624]:
feature_set = create_feature_set(all_songs_df, stat_cols, genre_weights_df)
feature_set.head()

C:\Users\ollie\AppData\Local\Temp\ipykernel_28296\823317900.py:13: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['genres'].explode()
C:\Users\ollie\AppData\Local\Temp\ipykernel_28296\823317900.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final['id']=df['id'].values


,21st century classical,abstract beats,abstract hip hop,acoustic guitar cover,acoustic pop,acoustic punk,acoustic rock,adult standards,aesthetic rap,african-american classical,afrobeat,afrofuturism,afropop,alaska indie,album rock,alt z,alternative dance,alternative emo,alternative hip hop,alternative metal,alternative pop rock,alternative r&b,alternative rock,ambeat,ambient,ambient folk,ambient pop,american 21st century classical,american modern classical,american orchestra,anime,anime lo-fi,anime piano,anime rap,anime rock,anime score,ann arbor indie,anthem emo,anti-folk,antiviral pop,argentine alternative rock,argentine hip hop,argentine indie,argentine indie rock,argentine rock,art pop,art punk,art rock,asian american hip hop,atl hip hop,atl trap,atlanta indie,aussietronica,austindie,australian alternative pop,australian alternative rock,australian dance,australian electropop,australian garage punk,australian indie,australian indie folk,australian pop,australian psych,australian r&b,avant-garde jazz,background jazz,background music,background piano,balalaika,ballet class,baltimore indie,bangalore indie,bangladeshi indie,barbadian pop,barockinterpreten,baroque,baroque ensemble,baroque pop,baroque singing,baroque violin,bases de freestyle,bass trap,bath indie,battle rap,bay area indie,beatlesque,bebop,bedroom pop,bedroom soul,belarusian indie,belgian hip hop,belgian singer-songwriter,bergen indie,big band,big beat,birmingham grime,bitpop,blues rock,bossa nova,bossa nova jazz,boston hip hop,boston indie,boston rock,bow pop,boy band,braindance,brazilian classical,brazilian indie,brazilian jazz,brazilian post-rock,brazilian rock,breakcore,brighton indie,brill building pop,brisbane indie,british alternative rock,british choir,british classical piano,british dance band,british indie rock,british invasion,british jazz,british math rock,british modern classical,british orchestra,british post-rock,british singer-songwriter,british soul,britpop,bronx hip hop,brooklyn drill,brooklyn indie,brostep,bubblegrunge,bubblegum pop,canadian americana,canadian contemporary country,canadian contemporary r&b,canadian electropop,canadian hip hop,canadian indie,canadian metal,canadian modern jazz,canadian pop,canadian psychedelic,canadian rock,canadian singer-songwriter,canadian trap,candy pop,caucasian classical piano,cello,chakra,chamber choir,chamber orchestra,chamber pop,chamber psych,channel islands indie,chanson,charlotte nc indie,chicago indie,chicago punk,chicago rap,chicago soul,chilean indie,chill beats,chill guitar,chill r&b,chillhop,chillsynth,chillwave,chinese hip hop,chinese traditional,chiptune,choral,christian lo-fi,city pop,classic anime,classic canadian rock,classic city pop,classic country pop,classic garage rock,classic girl group,classic j-pop,classic j-rock,classic japanese jazz,classic rock,classic soul,classic soundtrack,classical,classical accordion,classical cello,classical clarinet,classical countertenor,classical era,classical guitar,classical organ,classical performance,classical piano,classify,collage pop,colombian pop,comedy rap,comic,complextro,compositional ambient,connecticut indie,conscious hip hop,contemporary choir,contemporary jazz,contemporary post-bop,contemporary r&b,contemporary vocal jazz,cool jazz,corrido,corridos tumbados,cosmic post-rock,country rock,cowboy western,cyberpunk,czech electronic,dance pop,dance rock,dance-punk,dancehall,dancehall queen,danish pop,dark cabaret,dark pop,dark post-punk,dark synthpop,dark trap,dark wave,darksynth,dc indie,deathcore,deconstructed club,deep adult standards,deep east coast hip hop,deep gothic post-punk,deep idm,denpa-kei,denton tx indie,desert blues,digital hardcore,disco,disco house,disney,dixieland,diy emo,djent,dmv rap,doo-wop,double drumming,doujin,downtempo,downtempo bass,dream pop,dream smp,dreamo,dreampunk,drift,drift phonk,drill,drill and bass,drone,dunedin indie,dutch dnb,dutch pop,early modern classical,early music,early music choir,early music ensemble,e

In [625]:
def summarise_playlist(playlist_df, weight_decay, not_playlist=False):
    """
    Summarize a user's playlist into a single vector

    Parameters: 
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    if not_playlist:
        return playlist_df.sum(axis = 0).drop('id')
    else:
        playlist_feature_set = playlist_df.sort_values('added_at',ascending=False)
        most_recent_date = playlist_feature_set['added_at'][0]
        for ix, row in playlist_feature_set.iterrows():
            playlist_feature_set.loc[ix,'months_from_recent'] = int((pd.to_datetime(most_recent_date) - pd.to_datetime(row['added_at'])).days / 30)
        playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_decay ** (-x))
        playlist_feature_set_weighted = playlist_feature_set.copy()
        playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
        playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    return playlist_feature_set_weighted_final.mean(axis = 0)

In [626]:
def find_cosine_similarity(playlist_summary, wasabia_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_summary (pandas series): summarized playlist feature
        wasabia_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    wasabia_df = wasabia_features[['id']]
    wasabia_df['sim'] = cosine_similarity(wasabia_features.drop('id', axis = 1).values, playlist_summary.values.reshape(1, -1))[:,0]
    non_playlist_df_top_20 = wasabia_df.sort_values('sim',ascending = False)
    # non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_20

In [627]:
def find_similarity(feature_set, library_songs_length, wasabia_songs_length, wasabia_songs_df):
	library_features_df = feature_set.head(library_songs_length)
	wasabia_features_df = feature_set.tail(wasabia_songs_length).drop(['added_at'], axis = 1)

	summarised_library_features = summarise_playlist(library_features_df, 1.09)

	wasabia_cosine_similarity = find_cosine_similarity(summarised_library_features, wasabia_features_df)

	wasabia_songs_df = wasabia_songs_df.set_index('id')

	wasabia_songs_df['sim'] = wasabia_cosine_similarity.set_index('id')['sim']

	return wasabia_songs_df.sort_values('sim',ascending = False)

In [628]:
find_similarity(feature_set, library_songs_length, wasabia_songs_length, wasabia_songs_df).drop(['added_at', 'is_local', 'track', 'release_date'], axis=1)

,genres,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,sim
id,,,,,,,,,,,,,,,,
1jhYzm3nJOUuoQPQHNeqMk,[progressive post-hardcore],Spanish Bay,0.289,0.850,4,-5.789,1,0.0449,0.000140,0.871000,0.4600,0.2060,100.032,219349,4,0.648528
0H7sridQoXwfKbnUNBdhrb,[japanese alternative pop],YAMABIKO,0.737,0.936,2,-4.171,0,0.0993,0.277000,0.000000,0.0841,0.7420,105.055,217707,4,0.647710
1z9t09cCzfOz9R8UYjirt8,[indie rockism],Perspective,0.616,0.682,7,-8.024,1,0.0275,0.000282,0.001450,0.2440,0.7770,124.981,236587,4,0.647459
2DVR5wEXbauarRJhluXAZs,"[instrumental math rock, instrumental rock, ma...",nero,0.255,0.848,6,-4.553,0,0.0343,0.015900,0.892000,0.0997,0.5540,156.959,397800,4,0.646760
3R4b72wiBM1sB6V0i8IHaA,[],Funck,0.750,0.510,1,-7.966,1,0.0618,0.034100,0.204000,0.0768,0.7580,95.017,281790,4,0.635266
1xjaptI72LJvNF69LzEVaC,"[british math rock, math pop, post-rock]",Matador,0.413,0.681,0,-6.622,1,0.0888,0.020300,0.918000,0.3130,0.1220,100.995,366341,4,0.634399
5eltlS3CM4xGNkL6VhpTDm,"[alternative rock, anti-folk, baroque pop, can...",The Mariner's Revenge Song,0.391,0.581,9,-5.481,0,0.0901,0.250000,0.000000,0.0861,0.4590,83.587,525933,4,0.630691
0p4I9lDcIi65YzhKvC2QEc,[],Beginning of the End,0.669,0.581,8,-10.805,1,0.0443,0.231000,0.357000,0.1760,0.6320,93.050,244413,4,0.625527
7DcVWzeud5tqtNTZKQWvhz,"[japanese guitar, japanese math rock]",illusory sense,0.556,0.384,1,-10.490,0,0.0281,0.771000,0.001380,0.1390,0.1370,99.984,228364,3,0.619719


Note to self:

Playlist -> Tracks 

-> Get features

-> Get artists -> Genres

Associate features + genres with track!!